In [1]:
from pyspark import SparkContext
import pandas as pd

sc = SparkContext("local", "diabetes")

21/08/25 23:22:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/08/25 23:22:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
data = pd.read_csv('diabetes.csv')
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [3]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [4]:
sparkDF = sqlContext.createDataFrame(data)
sparkDF.printSchema()
sparkDF.show()

root
 |-- Pregnancies: long (nullable = true)
 |-- Glucose: long (nullable = true)
 |-- BloodPressure: long (nullable = true)
 |-- SkinThickness: long (nullable = true)
 |-- Insulin: long (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: long (nullable = true)
 |-- Outcome: long (nullable = true)



+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|
|          3|     78|           50|           32|     88|31.0|                   0.248| 26|      1|


In [21]:
sparkDF = sparkDF.filter(sparkDF['BloodPressure'] > 0)
sparkDF = sparkDF.filter(sparkDF['SkinThickness'] > 0)
sparkDF = sparkDF.filter(sparkDF['Insulin'] > 0)

sparkDF = sparkDF.withColumnRenamed('Outcome', 'label')
sparkDF.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-----+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|label|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-----+
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|    0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|    1|
|          3|     78|           50|           32|     88|31.0|                   0.248| 26|    1|
|          2|    197|           70|           45|    543|30.5|                   0.158| 53|    1|
|          1|    189|           60|           23|    846|30.1|                   0.398| 59|    1|
|          5|    166|           72|           19|    175|25.8|                   0.587| 51|    1|
|          0|    118|           84|           47|    230|45.8|                   0.551| 31|    1|
|          1|    103

In [22]:
split = sparkDF.randomSplit([0.7,0.3])
trainDF = split[0]
testDF = split[1]

In [23]:
from pyspark.ml.feature import VectorAssembler

va = VectorAssembler(inputCols=['Pregnancies', 'Glucose', 'BloodPressure', \
                                'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'] \
                     , outputCol='features')

In [24]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier()

In [25]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[va, rf, ])

In [26]:
model = pipeline.fit(trainDF)

In [30]:
prediction = model.transform(testDF)

selected = prediction.select(['label', 'prediction'])

In [32]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol='label', \
                                              predictionCol='prediction',\
                                              metricName="accuracy")
accuracy = evaluator.evaluate(selected)
accuracy

0.7889908256880734

In [34]:


from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

paramGrid = ParamGridBuilder() \
                .addGrid(rf.maxDepth, [3, 10, 20, 30]) \
                .addGrid(rf.numTrees, [5, 10, 20, 30]) \
                .build()

cross = CrossValidator(estimator=pipeline, \
                       estimatorParamMaps=paramGrid, \
                       evaluator=BinaryClassificationEvaluator())

cvmodel = cross.fit(trainDF)

21/08/25 23:46:24 WARN CacheManager: Asked to cache already cached data.
21/08/25 23:46:24 WARN CacheManager: Asked to cache already cached data.


In [35]:
prediction = cvmodel.transform(testDF)

In [37]:
selected = prediction.select(['label', 'prediction'])
selected.show()

+-----+----------+
|label|prediction|
+-----+----------+
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    1|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    1|       1.0|
|    1|       0.0|
|    1|       1.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 20 rows



In [38]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol='label', \
                                              predictionCol='prediction',\
                                              metricName="accuracy")
accuracy = evaluator.evaluate(selected)
accuracy

0.7889908256880734

In [40]:
from pyspark.ml.feature import StringIndexer

df = sqlContext.createDataFrame( \
         [(0, "a"), (1, "b"), (2, "c"), (3, "a"), (4, "a"), (5, "c")], ["id", "categoria"] \
    )

indexer = StringIndexer(inputCol="categoria", outputCol="categoriaIndex")
df = indexer.fit(df).transform(df)
df.show()

+---+---------+--------------+
| id|categoria|categoriaIndex|
+---+---------+--------------+
|  0|        a|           0.0|
|  1|        b|           2.0|
|  2|        c|           1.0|
|  3|        a|           0.0|
|  4|        a|           0.0|
|  5|        c|           1.0|
+---+---------+--------------+

